导入数据

In [1]:
import pandas as pd
import yagmail
import datetime

###############################################只需要修改filename即可，确定好datainputpath和dataoutputpath这两个文件夹与该py文件的相对位置
filename = "杭州 20240201_20240301 39769 条.csv"
print(filename)
###############################################注意命名格式是否正确 与csv的名字对应好
datainputpath = "data/杭州按月分类文件_201901_202406/"
dataoutputpath = "data/杭州按月分类文件_201901_202406_情绪值/"
########################### 自己设置区 ###############################
emailhost = "smtp.qq.com"
emailname = "473161189@qq.com"
emailpassword = "hspxxpupkwowbggg"  # 授权码 需要到邮箱的设置里面 开启授权
#####################################################################

# 发送者的邮箱名和授权码以及host
yag = yagmail.SMTP(user=emailname, password=emailpassword, host=emailhost)
# 被发送的邮箱名和内容
yag.send(
    to=emailname, subject="开始", contents="爬取文本信息\n" + "运行时间" + f"{filename}"
)

data = pd.read_csv(datainputpath + filename)

# filename=r'data\上海2019_2023年按月分类文件_情绪值\20210401_20210501 46859 条_Baidu.csv'
# data=pd.read_csv(filename)
tailname = "_Baidu.csv"  # 处理提取sentiment值后的文件

print("数据行数 ", len(data))  # 数据的行数
data.sample(2)  # 查看2

杭州 20240201_20240301 39769 条.csv
数据行数  39769


,mid,lat,lng,user_name,user_link,verify_typ,weibo_link,publish_time,content,image_urls,video_preview_url,forward_num,comment_num,like_num,phone_type,content_location_name,content_location_url,wgs_lng,wgs_lat
767,4996675099955997,30.257709,120.060843,CC的玛吉阿米,https://weibo.com/u/6930245748,没有认证,https://weibo.com/6930245748/NEyXzFM0B,2024-02-01 18:34,十里洲 ｜ 红梅<br><br>🌧️<br><br>霏霏丝雨落<br><br>袅袅浸梅香<b...,https://wx1.sinaimg.cn/large/007z0Bvegy1hme56q...,NaN,0,0,0,iPhone 11,杭州·杭州西溪十里芳菲主题度假村落,https://weibo.com/p/100101B2094254DA6CA2F5459A,120.056036,30.260012
22419,5002437523933040,30.287459,120.153576,泠七弦松风寒,https://weibo.com/u/7777316257,没有认证,https://weibo.com/7777316257/O11pugva8,2024-02-17 16:11,今日碎片 2杭州,https://wx3.sinaimg.cn/large/008ukPeNly1hmwj3o...,NaN,0,0,0,三星Galaxy S24 Ultra,杭州,https://weibo.com/p/1001018008633010000000000,120.148876,30.289784


使用百度NLP API进行情感分析 suspect better

In [2]:
import requests
import json

import pandas as pd 
key=pd.read_csv("data\key\XinBaiduAccesskey2.csv")
# key的读取 
API_KEY = key['API_KEY'][0]
SECRET_KEY = key['SECRET_KEY'][0]

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

def baidu_nlp(text):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token=" + get_access_token()
    params = {
        'access_token': 'your_access_token',
        'text': text
    }
    headers = {'Content-Type': 'application/json','Accept': 'application/json'}
        
    response = requests.request("POST",url, data=json.dumps(params), headers=headers)
    
    result = json.loads(response.text)
        
    result=response.json()

    for item in result['items']:
        confidence=item['confidence']
        negative_prob=item['negative_prob']
        positive_prob=item['positive_prob']
        sentiment=item['sentiment']
    
    return confidence, negative_prob, positive_prob,sentiment
    


In [3]:
# text="你好, 我是一名学生, 我很开心, 你呢？"
# # text="谭鸭血"

# c,n,p,s=baidu_nlp(text)
# print("confidence",c,"\nnegative_prob",n,"\npositive_prob",p,"\nsentiment",s)

In [4]:
import pandas as pd
import re
import jieba
import pandas as pd
import numpy as np
import datetime
from time import sleep
import random
import csv
import time
from tqdm import tqdm

print(filename)

def clean_content(content, place):
    # 去除地名
    content = content.replace(place, "")
    # 去除一些关键词
    content = (
        content.replace("分享图片", "")
        .replace("分享视频", "")
        .replace("微博视频", "")
        .replace("的微博视频", "")
        .replace("网页链接", "")
        .replace("超话", "")
        .replace("新浪图片", "")
        .replace("<br>", "")
        .replace("的秒拍视频","")
    )
    # 去除英文
    content = re.sub(r"[a-zA-Z]+", "", content)
    content = (
        re.sub(r"\d+", "", content).replace(" ", "").replace(".", "").replace("_", "")
    )
    # 去除所有非中文符号
    content = re.sub(r"[^\u4e00-\u9fa5]", "", content)
    # 去除空白字符
    content = re.sub(r"\s+", "", content)
    return content


time_start = time.time()
print("开始 ", datetime.datetime.now())

# 对content列进行清洗 由对应的地名在文本中去除content文本中的内容
data["clean_content"] = data.apply(
    lambda row: clean_content(row["content"], row["content_location_name"]), axis=1
)
print("清洗完成 " + str(len(data)))

# 如果文本为空 去掉该行
data = data[data["clean_content"] != ""]
print("去除空白行完成 " + str(len(data)))

# 保留文本字数大于3的和小于512的文本   百度是512字、腾讯是200字
data = data[
    (data["clean_content"].apply(lambda x: len(x) < 512))
    & (data["clean_content"].apply(lambda x: len(x) > 3))
]
print("保留字数大于3完成 " + str(len(data)))

# 对清洗后的文本 进行 分词 放入jieba_cut列中
data["jieba_cut"] = data["clean_content"].apply(lambda x: jieba.lcut(x))
print("分词完成 " + str(len(data)))


def process_row(row):
    try:
        # 如果是nan 这里被标注的if是后续的处理 不用管
        # if pd.isna(row['baidu_confidence']):
        confidence, negative, positive, sentiment = baidu_nlp(row["clean_content"])
        # 休息几秒钟 确保正确跑完
        # sleep(0.04*random.randint(2,3)) # 0.04秒到0.06秒 之间随机休息比较好
        sleep(0.2)  # 大量测试 0.1算比较合适 可以适当减少
        return confidence, negative, positive, sentiment
    # else:
    # return row['baidu_confidence'], row['baidu_negative'], row['baidu_positive'], row['baidu_sentiment']
    except:
        print(row["publish_time"] +' '+ row["clean_content"])
        print("百度API调用失败")
        return np.nan, np.nan, np.nan, np.nan


tqdm.pandas()
results = data.progress_apply(process_row, axis=1)
# results = data.apply(process_row, axis=1)
try:
    (
        data["baidu_confidence"],
        data["baidu_negative"],
        data["baidu_positive"],
        data["baidu_sentiment"],
    ) = zip(*results)
    # 保存
    # with open(filename, 'w', encoding='utf-8-sig', newline='') as file:
    with open(
        dataoutputpath + filename.split(".")[0] + tailname,
        "w",
        encoding="utf-8-sig",
        newline="",
    ) as file:
        writer = csv.writer(file)
        writer.writerow(data.columns)  # 写入列名
        for row in data.itertuples(index=False):
            writer.writerow(row)

    print(data.head(5))
    time_end = time.time()
    print(
        filename,
        "\n本次运行总共耗时：",
        time_end - time_start,
        "\n即：",
        (time_end - time_start) / 60,
        "分钟",
        "\n即：",
        (time_end - time_start) / 3600,
        "小时",
    )
    print("结束", datetime.datetime.now())
    yag.send(
        to=emailname,
        subject="结束",
        contents=filename + "爬取文本信息\n" + "运行时间" + f"{time_end-time_start}",
    )
except:
    print("百度API调用失败，将results保存后退出")
    results.to_csv("error" + filename, index=False, encoding="utf-8-sig")
print("情感分析完成")

杭州 20240201_20240301 39769 条.csv
开始  2024-08-27 22:58:43.598256


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\name\AppData\Local\Temp\jieba.cache


清洗完成 39769
去除空白行完成 35841
保留字数大于3完成 32580


Loading model cost 0.577 seconds.
Prefix dict has been built successfully.


分词完成 32580


...

2024-02-21 21:29 春节档回忆
百度API调用失败


 76%|███████▌  | 24776/32580 [4:55:52<13:44:21,  6.34s/it]

2024-02-23 13:42 今日份的减脂餐蔬菜和主食没有蛋白质超了意式黑松露生牛肉薄片和香煎红酒鹅肝优秀眼肉丰腴也不错拉花这么好看有点舍不得喝杭州美食随手拍不可辜负的美食我的美食日记减脂
百度API调用失败


100%|██████████| 32580/32580 [6:30:52<00:00,  1.39it/s]   


                mid        lat         lng    user_name  \
0  4996394790160877  30.287459  120.153576  Richard_DWH   
1  4996394996207807  30.316310  120.137210       微尘儿之于穹   
2  4996395075109260  30.253114  120.163342       -Gmxx-   
3  4996395037623958  30.240209  120.206498  落叶弥生-金様sama   
4  4996395442637788  30.225363  120.125068        钦亲亲宝贝   

                        user_link verify_typ  \
0  https://weibo.com/u/7724854678       黄V认证   
1  https://weibo.com/u/5970128933       没有认证   
2  https://weibo.com/u/5951684932       没有认证   
3    https://weibo.com/jinpinjing       没有认证   
4  https://weibo.com/u/2994587062       没有认证   

                               weibo_link      publish_time  \
0  https://weibo.com/7724854678/NErFt0FQN  2024-02-01 00:00   
1  https://weibo.com/5970128933/NErFNq2VV  2024-02-01 00:00   
2  https://weibo.com/5951684932/NErFVlr9q  2024-02-01 00:01   
3  https://weibo.com/1672977045/NErFRvZl4  2024-02-01 00:01   
4  https://weibo.com/2994587062/NErGwb4cY

In [5]:
# import requests
# import json


# def main():
        
#     url = "https://aip.baidubce.com/oauth/2.0/token?client_id=529ItGYf8DuvhF8OdKUmCU8q&client_secret=CW3ylLBnSMzOWJKhWt4JXaIgljtKCpku&grant_type=client_credentials"
    
#     payload = json.dumps("")
#     headers = {
#         'Content-Type': 'application/json',
#         'Accept': 'application/json'
#     }
    
#     response = requests.request("POST", url, headers=headers, data=payload)
    
#     print(response.text)
    

# if __name__ == '__main__':
#     main()


百度云NLP的情感倾向分析

In [6]:
# import requests
# import json

# API_KEY = "529ItGYf8DuvhF8OdKUmCU8q"
# SECRET_KEY = "CW3ylLBnSMzOWJKhWt4JXaIgljtKCpku"

# def main():
        
#     url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token=" + get_access_token()
    
#     payload = json.dumps({
#         # "text": "欢庆下周挺进病房🙃希望这不是最后的狂欢🤓🤓 2上海·胡桃里音乐酒馆(南洋1931店) ​​​​"
#         "text":"顶天立地今天的舞蹈秀太赞了👍 2上海·水岸秀苑"
#     })
#     headers = {
#         'Content-Type': 'application/json',
#         'Accept': 'application/json'
#     }
    
#     response = requests.request("POST", url, headers=headers, data=payload)
    
#     print(response.text)

#     jd=json.loads(response.text)
#     jd=response.json()
#     print(jd)
#     for item in jd['items']:
#         print(item['confidence'])
#         print(item['negative_prob'])
#         print(item['positive_prob'])
#         print(item['sentiment'])
    
    

# def get_access_token():
#     """
#     使用 AK，SK 生成鉴权签名（Access Token）
#     :return: access_token，或是None(如果错误)
#     """
#     url = "https://aip.baidubce.com/oauth/2.0/token"
#     params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
#     return str(requests.post(url, params=params).json().get("access_token"))

# if __name__ == '__main__':
#     main()


In [7]:
# import requests
# import json

# API_KEY = "529ItGYf8DuvhF8OdKUmCU8q"
# SECRET_KEY = "CW3ylLBnSMzOWJKhWt4JXaIgljtKCpku"

# def main():
        
#     url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/emotion?charset=UTF-8&access_token=" + get_access_token()
    
#     payload = json.dumps("")
#     headers = {
#         'Content-Type': 'application/json',
#         'Accept': 'application/json'
#     }
    
#     response = requests.request("POST", url, headers=headers, data=payload)
    
#     print(response.text)
    

# def get_access_token():
#     """
#     使用 AK，SK 生成鉴权签名（Access Token）
#     :return: access_token，或是None(如果错误)
#     """
#     url = "https://aip.baidubce.com/oauth/2.0/token"
#     params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
#     return str(requests.post(url, params=params).json().get("access_token"))

# if __name__ == '__main__':
#     main()


使用百度NLP API（SDK方法）进行情感分析

In [8]:
# from aip import AipNlp
# import pandas as pd

# key=pd.read_csv('data\key\BaiduAccesskey.csv')
# """ 你的 APPID AK SK """
# APP_ID = str(key['APP_ID'][0])
# API_KEY = key['API_KEY'][0]
# SECRET_KEY = key['SECRET_KEY'][0]

# # APP_ID
# client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

# # text = "苹果是一家伟大的公司"
# text= "谭鸭血a"

# """ 调用情感倾向分析 """
# k=client.sentimentClassify(text)
# print(k)

# def baidu_nlp(text):
    
#     result=client.sentimentClassify(text)
#     for item in result['items']:
#         confidence=item['confidence']
#         negative=item['negative_prob']
#         positive=item['positive_prob']
#         sentiment=item['sentiment']

#     return confidence,negative,positive,sentiment
    


统计文件中标出的数据条数的总数

In [9]:
# import os

# def count_total_records(folder_path):
#     total_records = 0
    
#     # 遍历文件夹中的所有文件
#     for filename in os.listdir(folder_path):
#         if filename.endswith('.csv'):
#             # 提取文件名中的记录条数
#             parts = filename.split()
#             if len(parts) >= 3 and parts[-2].isdigit():
#                 records = int(parts[-2])
#                 total_records += records
    
#     return total_records

# # 使用示例
# folder_path = 'data\上海2019_2023年按月分类文件'
# total_records = count_total_records(folder_path)
# print(f'总记录条数: {total_records}')
